Making a keras model.

In [146]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import SimpleRNN, Reshape
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import h5py
from numba import jit
from sklearn.model_selection import train_test_split

### run this if you're wanting to just use d stats

In [104]:
dat=h5py.File("databases/slider_quarts_bigger.hdf5",mode="r")
@jit
def dstat(mat):
    '''
    input a 2d square site count matrix, output a d statistic (float)
    '''
    abba = (mat[1,4]+mat[2,8]+mat[3,12]+mat[4,1]+mat[6,9]+mat[7,13]+
            mat[8,2]+mat[9,6]+mat[11,14]+mat[12,3]+mat[13,7]+mat[14,11])
    baba = (mat[1,1]+mat[2,2]+mat[3,3]+mat[4,4]+mat[6,6]+mat[7,7]+
            mat[8,8]+mat[9,9]+mat[11,11]+mat[12,12]+mat[13,13]+mat[14,14])
    return(float(abba-baba)/(abba+baba))
def dstat3d(simcat_arr):
    dstats = np.zeros((len(simcat_arr)))
    counter = 0
    for layer in simcat_arr:
        try:
            dstats[counter] = dstat(layer)
        except:
            # if division by zero error, then return an array of zeros that you can search for later
            dstats = np.zeros((len(simcat_arr)))
            break
        counter += 1
    return(dstats)
all_dstats = np.zeros(dat['counts'].shape[0:2])
for _ in range(len(dat['counts'])):
    all_dstats[_]=dstat3d(dat['counts'][_])
failed = list()
for _ in range(len(all_dstats)):
    if np.alltrue(all_dstats[_] == 0):
        failed.append(_)
np.array(failed)
ind = np.ones((len(all_dstats)), bool)
ind[failed] = False
sources=dat['admix_sources'][:][ind]
targets=dat['admix_targets'][:][ind]
combo=np.hstack([sources,targets])

class_ids=np.array([str(x) for x in combo])
id_dict=dict(enumerate(np.unique(class_ids)))
inv_dict = {v: k for k, v in id_dict.iteritems()}
class_ids_int = np.zeros(class_ids.shape,dtype=np.int32)
counter = 0
for str_id in class_ids:
    class_ids_int[counter] = inv_dict[str_id]
    counter += 1

X = all_dstats[ind]
y = class_ids_int

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [107]:
combo

array([[1, 2],
       [1, 2],
       [1, 2],
       ...,
       [5, 3],
       [5, 3],
       [5, 3]], dtype=uint8)

### For raw data, run this:

In [185]:
dat=h5py.File("databases/bounded_05_4.hdf5")

In [186]:
sources=dat['admix_sources']
targets=dat['admix_targets']
combo=np.hstack([sources,targets])
newcombo = np.sort(combo,axis=1)
class_ids=np.array([str(x) for x in newcombo])
id_dict=dict(enumerate(np.unique(class_ids)))
inv_dict = {v: k for k, v in id_dict.iteritems()}
class_ids_int = np.zeros(class_ids.shape,dtype=np.int32)
counter = 0
for str_id in class_ids:
    class_ids_int[counter] = inv_dict[str_id]
    counter += 1
y = class_ids_int

In [187]:
def flattendb(counts):
    newshape=reduce(lambda x,y: x*y, counts[0].shape)
    X = np.zeros((counts.shape[0],newshape))
    for i in range(len(counts)):
        X[i] = counts[i].ravel()
    return(X)

In [188]:
X = flattendb(dat['counts'])

In [189]:
trainX, testX, trainY, testY = train_test_split(X, y,
	test_size = .2, random_state = 42)

In [190]:
encoding_trainY = np_utils.to_categorical(trainY)
encoding_testY = np_utils.to_categorical(testY)

In [191]:
# Creating a model
model = Sequential()
model.add(Dense(1024, input_dim=256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(9, activation='softmax'))

In [88]:
model = Sequential()
model.add(Dense(512, input_dim=256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Reshape((16, 16)))
model.add(SimpleRNN(9)) 

model.add(Dense(9, activation='softmax'))
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss="mean_squared_error", optimizer="rmsprop")

In [193]:
# Compiling model
#adam = Adam(lr=0.5, beta_1=0.9, beta_2=0.99, epsilon=.01)
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

In [194]:
# Training a model
model.fit(trainX, encoding_trainY, epochs=30, batch_size=100,verbose=True)

Epoch 1/30
14400/14400 [==============================] - 6s 440us/step - loss: 0.9268 - acc: 0.6585
Epoch 2/30
14400/14400 [==============================] - 5s 313us/step - loss: 0.7340 - acc: 0.7185
Epoch 3/30
14400/14400 [==============================] - 6s 397us/step - loss: 0.7240 - acc: 0.7204
Epoch 4/30
14400/14400 [==============================] - 5s 321us/step - loss: 0.6927 - acc: 0.7320
Epoch 5/30
14400/14400 [==============================] - 4s 286us/step - loss: 0.6808 - acc: 0.7305
Epoch 6/30
14400/14400 [==============================] - 5s 358us/step - loss: 0.6727 - acc: 0.7355
Epoch 7/30
14400/14400 [==============================] - 5s 367us/step - loss: 0.6729 - acc: 0.7369
Epoch 8/30
14400/14400 [==============================] - 6s 390us/step - loss: 0.6678 - acc: 0.7345
Epoch 9/30
14400/14400 [==============================] - 5s 342us/step - loss: 0.6620 - acc: 0.7402
Epoch 10/30
14400/14400 [==============================] - 5s 319us/step - loss: 0.6645 - a

In [11]:
model.fit(trainX, encoding_trainY, epochs=50, batch_size=20,verbose=True)

Epoch 1/50
7360/7360 [==============================] - 5s 708us/step - loss: 0.7001 - acc: 0.7443
Epoch 2/50
7360/7360 [==============================] - 5s 698us/step - loss: 0.4407 - acc: 0.8129
Epoch 3/50
7360/7360 [==============================] - 5s 706us/step - loss: 0.3967 - acc: 0.8265
Epoch 4/50
7360/7360 [==============================] - 5s 710us/step - loss: 0.3489 - acc: 0.8385
Epoch 5/50
7360/7360 [==============================] - 5s 718us/step - loss: 0.3452 - acc: 0.8368
Epoch 6/50
7360/7360 [==============================] - 5s 707us/step - loss: 0.3789 - acc: 0.8281
Epoch 7/50
7360/7360 [==============================] - 5s 699us/step - loss: 0.3322 - acc: 0.8442
Epoch 8/50
7360/7360 [==============================] - 5s 716us/step - loss: 0.3318 - acc: 0.8367
Epoch 9/50
7360/7360 [==============================] - 5s 718us/step - loss: 0.3097 - acc: 0.8461
Epoch 10/50
7360/7360 [==============================] - 5s 707us/step - loss: 0.3350 - acc: 0.8446
Epoch 11/

In [12]:
# evaluate the model
scores = model.evaluate(testX, encoding_testY)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

2000/2000 [==============================] - 0s 201us/step

Accuracy: 41.55%


In [16]:
# evaluate the model
scores = model.evaluate(testX, encoding_testY)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

2000/2000 [==============================] - 4s 2ms/step

Accuracy: 42.45%


In [20]:
# evaluate the model
scores = model.evaluate(testX, encoding_testY)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

2000/2000 [==============================] - 3s 1ms/step

Accuracy: 44.50%


In [24]:
# evaluate the model
scores = model.evaluate(testX, encoding_testY)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

2000/2000 [==============================] - 1s 727us/step

Accuracy: 41.50%


In [28]:
# evaluate the model
scores = model.evaluate(testX, encoding_testY)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

2000/2000 [==============================] - 2s 897us/step

Accuracy: 41.40%


In [43]:
# evaluate the model
scores = model.evaluate(testX, encoding_testY)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

1840/1840 [==============================] - 2s 1ms/step

Accuracy: 86.03%


In [195]:
# evaluate the model
scores = model.evaluate(testX, encoding_testY)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

3600/3600 [==============================] - 3s 705us/step

Accuracy: 73.83%


In [157]:
# evaluate the model
scores = model.evaluate(testX, encoding_testY)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

3600/3600 [==============================] - 1s 281us/step

Accuracy: 73.94%


In [158]:
from sklearn.metrics import confusion_matrix
from toyplot import matrix

In [159]:
confusion_matrix(np.argmax(encoding_testY,axis=1),np.argmax(model.predict(testX),axis=1))

array([[233,  25,   0,  13,   0,   0,  75,   4,  20],
       [  7, 357,   0,  37,   0,   0,  27,   0,   0],
       [  0,   1, 319,   2,  17,   2,   1,  45,   3],
       [  8,  63,   0, 291,   0,   0,  25,   0,   7],
       [  2,   1,  31,   0, 310,   4,   2,  32,   2],
       [  6,   0,   7,   0,   2, 354,   2,  14,   6],
       [ 89,  68,   0,  40,   0,   0, 164,   1,  23],
       [  1,   0,  24,   0,  18,  11,   3, 337,  24],
       [ 74,   4,   0,   4,   1,   6,  34,  20, 297]])

{'[0 1]': 0,
 '[0 2]': 1,
 '[0 3]': 2,
 '[1 2]': 3,
 '[1 3]': 4,
 '[2 3]': 5,
 '[2 4]': 6,
 '[3 4]': 7,
 '[3 5]': 8}